In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, PeftModel, PeftConfig

In [ ]:
download_base_tokenizer = True

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
peft_model_id = "adapters/Orca-2-13b-4bit-finetune-on-ARC"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                                    quantization_config=bnb_config,
                                                    return_dict=True,
                                                    load_in_4bit=True,
                                                    device_map={"":0})

if download_base_tokenizer:
    base_model_name = "microsoft/Orca-2-13b"
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
else:
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
def open_arc_json(json_file):
    with open(json_file, "r") as f:
        task = json.load(f)
    return task


def pick_random_file(directory):
    subdirs = [
        d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))
    ]
    if not subdirs:
        json_files = [f for f in os.listdir(directory) if f.endswith(".json")]
        if json_files:
            index = random.randint(0, len(json_files) - 1)
            return os.path.join(directory, json_files[index])
    subdir = os.path.join(directory, random.choice(subdirs))
    for root, dirs, files in os.walk(subdir):
        json_files = [f for f in files if f.endswith(".json")]
        if json_files:
            index = random.randint(0, len(json_files) - 1)
            return os.path.join(root, json_files[index])
    return None


def get_random_ARC_evaluation_puzzle():
    directory = "data/step_2/evaluation_max_3_trains_1_test/"
    file_path = ""
    while not file_path.endswith(".json"):
        file_path = pick_random_file(directory)
    # file_path = "data/step_2/evaluation_augs/0a1d4ef_100.json"
    print(file_path)
    puzzle = open_arc_json(file_path)
    return puzzle

In [ ]:
# import generate_puzzles_v003 as gp
import re
import ast

# num_train_pairs = 3
# num_test_pairs = 1
# min_grid_dim = 8
# max_grid_dim = 30
# random_puzzle_json = gp.generate_random_puzzle_step_1(
#     num_train_pairs, num_test_pairs, min_grid_dim, max_grid_dim
# )

random_puzzle_json = get_random_ARC_evaluation_puzzle()
instruction, output = gp.format_instruction_and_output(random_puzzle_json)
print("\ninstruction\n", instruction)

user_message = instruction
query = f"<|im_start|>user\n{user_message}<|im_end|>"
encodeds = tokenizer(query, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to("cuda")

model.eval()
with torch.no_grad():
  generated_ids = model.generate(**model_inputs, max_new_tokens=4096, do_sample=False, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
result = decoded[0]
result = result[len(query) :]

match = re.search(r"Test_1_Output = \[\[.*?\]\]", result)
if match:
    array_string = match.group(0).replace("Test_1_Output = ", "")
    array = ast.literal_eval(array_string)
    print("\nResult")
    print(array)
    print("")
else:
    print("\nNo match found")
    print(result)
    print("")
    array = []
